In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
REPO_NAME = 'sewer-nfl'
import sys, os
cwd = str(os.getcwd())
repo_dir = cwd[:cwd.find(REPO_NAME)+len(REPO_NAME)]
sys.path.insert(0,repo_dir)

In [3]:
import pandas as pd
import nfl_data_py as nflreadr

In [4]:
from warehouse.utilities.elo import calculate_elo_metric

In [5]:
api_data = nflreadr.import_pbp_data(range(2021,2023))

2021 done.
2022 done.
Downcasting floats.


In [6]:
api_data = api_data[api_data['week']<=18]

In [7]:
from warehouse.pipelines.pbp.pbp import player_involvement, player_involved, players_involved

In [8]:
player_involvement_df = player_involvement(pbp_data=api_data)

In [9]:
player_involved(player_involvement_df=player_involvement_df,
                #play_id = 40.0,
                #game_id = '2021_01_ARI_TEN',
                player_id = '00-0032560')

,play_id,game_id,home_team,away_team,posteam,posteam_type,variable,value
1,40.0,2021_01_ARI_TEN,TEN,ARI,TEN,home,o1,00-0032560
17,451.0,2021_01_ARI_TEN,TEN,ARI,TEN,home,o1,00-0032560
27,701.0,2021_01_ARI_TEN,TEN,ARI,TEN,home,o1,00-0032560
35,885.0,2021_01_ARI_TEN,TEN,ARI,ARI,away,o1,00-0032560
56,1385.0,2021_01_ARI_TEN,TEN,ARI,TEN,home,o1,00-0032560
...,...,...,...,...,...,...,...,...
1248560,298.0,2022_18_ARI_SF,SF,ARI,SF,home,d2,00-0032560
1248629,1959.0,2022_18_ARI_SF,SF,ARI,SF,home,d2,00-0032560
1248641,2211.0,2022_18_ARI_SF,SF,ARI,SF,home,d2,00-0032560
1248654,2508.0,2022_18_ARI_SF,SF,ARI,SF,home,d2,00-0032560


In [61]:
meta_columns = [
    'game_id',
    'old_game_id',
    'season',
    'week',
    'home_team',
    'away_team',
    'posteam',
    'spread_line'
]

play_columns = [
    'play_id',
    'play_type',
    'desc',
    'yards_gained',
    'passer_player_id',
    'receiver_player_id',
    'rusher_player_id',
    'offense_players',
    'defense_players'
]

epa_columns = [
    'epa',
    'air_epa',
    'yac_epa'
]

In [67]:
# Player Id Lookup function
rosters = nflreadr.import_rosters(years=[2020,2022])

In [68]:
r_columns = [
    'season',
    'week',
    'team',
    'position',
    'player_name',
    'player_id'
]
rosters[r_columns]

,season,week,team,position,player_name,player_id
0,2020,17,ARI,DB,Johnathan Joseph,00-0024239
1,2020,17,ARI,DB,Patrick Peterson,00-0027943
2,2020,17,ARI,DB,Prince Amukamara,00-0027957
3,2020,17,ARI,DB,Dre Kirkpatrick,00-0029560
4,2020,17,ARI,DB,Chris Banjo,00-0029747
...,...,...,...,...,...,...
2636,2022,18,WAS,WR,Antonio Gandy-Golden,00-0036340
2637,2022,18,WAS,WR,Dyami Brown,00-0036626
2638,2022,18,WAS,WR,Dax Milne,00-0036671
2639,2022,18,WAS,WR,Jahan Dotson,00-0037741


In [131]:
from scipy.stats import zscore

In [169]:
# Use to reduce outlier positions
position_filter = ['QB','RB','WR','TE']

dataset_cols = meta_columns + [
    'play_id',
    'play_type',
    'yards_gained',
    'rusher_player_id',
    'epa'
]

epa_dataset = api_data[dataset_cols]

mask = (epa_dataset['play_type']=='run')

epa_dataset = epa_dataset.loc[mask].merge(
    rosters[[
        'player_name',
        'position',
        'player_id'
        ]],
    left_on = ['rusher_player_id'],
    right_on = ['player_id'],
    how = 'inner'
).drop_duplicates()

position_mask = (epa_dataset['position'].isin(position_filter))

epa_dataset = epa_dataset[position_mask]

epa_dataset['defteam'] = epa_dataset.apply(lambda x : x['home_team'] if x['away_team']==x['posteam'] else x['away_team'], axis = 1)
# Pre-elo datasets should be at the game / team / player level

order_cols = ['season','week'] # Used in ELO function
off_gb_cols = ['posteam','defteam','position','rusher_player_id','player_name'] # At what level are we AGGREGATING offensive performance
gb_cols_z = ['season','position'] # What is aggregate level for z score
def_gb_cols = ['posteam','defteam','position'] # Matched up against what level on defense to AGGREGATE to
perf_cols = ['epa'] # What are we assessing for ELO

off_epa_dataset = epa_dataset.groupby(order_cols + off_gb_cols)[perf_cols].sum().reset_index()
for col in perf_cols:
    off_epa_dataset[f'z_{col}'] = off_epa_dataset.groupby(order_cols + gb_cols_z)[col].transform(lambda x : zscore(x))
off_epa_dataset.drop(perf_cols, axis = 1, inplace = True)

def_epa_dataset = epa_dataset.groupby(order_cols + def_gb_cols)[perf_cols].sum().reset_index()
for col in perf_cols:
    def_epa_dataset[f'z_{col}'] = def_epa_dataset.groupby(order_cols + gb_cols_z)[col].transform(lambda x : zscore(x))
def_epa_dataset.drop(perf_cols, axis = 1, inplace = True)

# Don't need to use cumulative functions, because this is retroactive and we can use long term averages by position
#  epa_dataset[[f'cum_{col}' for col in perf_cols]] = epa_dataset.groupby(off_gb_cols_cum)[perf_cols].cumsum()
# epa_dataset['cum_n'] = epa_dataset.groupby(off_gb_cols_cum)[perf_cols[0]].cumcount()
# for col in perf_cols:
#     epa_dataset[f'cummean_{col}'] = epa_dataset[f'cum_{col}'] / (epa_dataset['cum_n'] + 1)
# # Calculate standard deviation
# epa_dataset.groupby(off_gb_cols_cum)['yards_gained'].expanding().std()

In [174]:
off_epa_dataset

,season,week,posteam,defteam,position,rusher_player_id,player_name,z_epa
0,2021,1,ARI,TEN,QB,00-0035228,Kyler Murray,-0.320083
1,2021,1,ARI,TEN,RB,00-0033553,James Conner,-0.120027
2,2021,1,ARI,TEN,RB,00-0034681,Chase Edmonds,1.566789
3,2021,1,ATL,PHI,QB,00-0026143,Matt Ryan,-0.640594
4,2021,1,ATL,PHI,RB,00-0030578,Cordarrelle Patterson,2.207344
...,...,...,...,...,...,...,...,...
4487,2022,18,WAS,DAL,RB,00-0032975,Jonathan Williams,-2.008978
4488,2022,18,WAS,DAL,RB,00-0034578,Reggie Bonnafon,-0.074378
4489,2022,18,WAS,DAL,RB,00-0036755,Jaret Patterson,-0.082603
4490,2022,18,WAS,DAL,TE,00-0037168,Armani Rogers,-1.412397


In [252]:
# Generate shell dataframe for ELO metrics

off_df = off_epa_dataset
def_df = def_epa_dataset

elo_df = off_df.merge(def_df, 
                      on = order_cols + def_gb_cols,
                      how = 'left')

# Note: Performance may not be equivalent between offense and defense, 
# since levels of aggregation may be different. All that matters is that
# the ELO multiplier is applied equivalently to both adjustments. Also, 
# remember performance will be inverse for defense

elo_df['off_elo'] = None
elo_df['def_elo'] = None

minimums = {col:elo_df[col].min() for col in order_cols}
elo_base = 2000

for col in ['off_elo','def_elo']:
    elo_df[col] = elo_df.apply(
        lambda row: \
            elo_base if list(minimums.values()) == [row[key] for key in minimums.keys()] \
                else None, 
        axis=1)

def elo_multiplier(off_elo, def_elo, power = 1.1, multiplier = 10):
    return (off_elo / def_elo) ** power * multiplier

import math as m

off_lock = ['posteam','position','rusher_player_id']
off_lookback = order_cols # ['season','week']

for index, row in elo_df.iterrows():
    off_elo = row['off_elo']
    def_elo = row['def_elo']
    # Grab prior off_elo and def_elo if not in minimums
    if m.isnan(off_elo):
        # Find team's prior order column - if not found, go back one order column
        # ie. if week 1 of a new season, have to go back to prior season (and get max of current week value)
        temp = elo_df.loc[
            (elo_df['position'] == row['position']) & \
            (elo_df['rusher_player_id'] == row['rusher_player_id']) & \
            (elo_df['season'] == row['season']) & \
            (elo_df['week'] == row['week'] - 1)
        ].squeeze()
        print(temp['off_elo_next'])
        print(temp)
        off_elo = temp['off_elo_next']

    off_elo_next = elo_multiplier(off_elo, def_elo) * row['z_epa_x']
    elo_df.at[index, 'off_elo_next'] = off_elo_next + row['off_elo']

1996.7991699083054
season                      2021
week                           1
posteam                      ARI
defteam                      TEN
position                      QB
rusher_player_id      00-0035228
player_name         Kyler Murray
z_epa_x                -0.320083
z_epa_y                -0.304799
off_elo                   2000.0
def_elo                   2000.0
off_elo_next          1996.79917
Name: 0, dtype: object
1998.7997298429489
season                      2021
week                           1
posteam                      ARI
defteam                      TEN
position                      RB
rusher_player_id      00-0033553
player_name         James Conner
z_epa_x                -0.120027
z_epa_y                 1.314033
off_elo                   2000.0
def_elo                   2000.0
off_elo_next          1998.79973
Name: 1, dtype: object
2015.6678885106282
season                       2021
week                            1
posteam                       ARI
def

ValueError: Incompatible indexer with Series

In [207]:
elo_df

,season,week,posteam,defteam,position,rusher_player_id,player_name,z_epa_x,z_epa_y,off_elo,def_elo
0,2021,1,ARI,TEN,QB,00-0035228,Kyler Murray,-0.320083,-0.304799,2000.0,2000.0
1,2021,1,ARI,TEN,RB,00-0033553,James Conner,-0.120027,1.314033,2000.0,2000.0
2,2021,1,ARI,TEN,RB,00-0034681,Chase Edmonds,1.566789,1.314033,2000.0,2000.0
3,2021,1,ATL,PHI,QB,00-0026143,Matt Ryan,-0.640594,-0.586282,2000.0,2000.0
4,2021,1,ATL,PHI,RB,00-0030578,Cordarrelle Patterson,2.207344,0.736142,2000.0,2000.0
...,...,...,...,...,...,...,...,...,...,...,...
4487,2022,18,WAS,DAL,RB,00-0032975,Jonathan Williams,-2.008978,-1.545616,NaN,NaN
4488,2022,18,WAS,DAL,RB,00-0034578,Reggie Bonnafon,-0.074378,-1.545616,NaN,NaN
4489,2022,18,WAS,DAL,RB,00-0036755,Jaret Patterson,-0.082603,-1.545616,NaN,NaN
4490,2022,18,WAS,DAL,TE,00-0037168,Armani Rogers,-1.412397,-1.412397,NaN,NaN
